## Data Mining Final
### Twitter data and stock analysis - Tesla

**Business Question:**

We would like to analyze Twitter data related to Tesla and its influence in Tesla Stock's price by using NLP and sentiment analysis.

### 0. installation and importation of library

In [30]:
#!pip install langdetect

In [79]:
import langdetect as ld
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re

### 1. Read Data and Exploration

#### Explore Tweet Dataframe

In [2]:
df_tweet = pd.read_csv("Hashtag Tesla Tweets.csv")

In [4]:
df_tweet.head(5)

,Date & Time,Profile Picture Link,Twitter ID,Tweet Text,Tweet Link
0,"April 10, 2022 at 07:44PM",http://pbs.twimg.com/profile_images/15120745...,@Jessica1988kk,"RT @invest_answers: Crypto news, #Bitcoin Whal...",https://twitter.com/Jessica1988kk/status/15131...
1,"April 10, 2022 at 07:45PM",http://pbs.twimg.com/profile_ s/87878355348773...,@JotaGe2014,#Tesla tiene récord de autos vendidos. Es impr...,https://twitter.com/JotaGe2014/status/15131737...
2,"April 10, 2022 at 07:45PM",http://pbs.twimg.com/profile_images/936422368...,@MmeCallas,RT @CottonCodes: 🐒 #love in my #MariaCallas I ...,https://twitter.com/MmeCallas/status/151317374...
3,"April 10, 2022 at 07:45PM",http://pbs.twimg.com/profile_images/146366591...,@BotSecx,RT @CottonCodes: 🐒 #love in my #MariaCallas I ...,https://twitter.com/BotSecx/status/15131737626...
4,"April 10, 2022 at 07:45PM",http://pbs.twimg.com/profile_images/111675859...,@agseh,RT @RupiReportero_: 🙆‍♂️🚘 Al que le robaron la...,https://twitter.com/agseh/status/1513173864829...


In [5]:
df_tweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152000 entries, 0 to 151999
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Date & Time           152000 non-null  object
 1   Profile Picture Link  152000 non-null  object
 2   Twitter ID            152000 non-null  object
 3   Tweet Text            152000 non-null  object
 4   Tweet Link            152000 non-null  object
dtypes: object(5)
memory usage: 5.8+ MB


In [15]:
number_tweet = df_tweet.groupby("Twitter ID")[["Tweet Text"]].count().sort_values(by = "Tweet Text", ascending = False)

In [16]:
number_tweet.head(5)

,Tweet Text
Twitter ID,
@TeslaradarB,7411
@TeslaHourly,1354
@amazingvids3,1177
@JNHaldemann,1113
@Zxcxz_xyz,1035


In [22]:
df_tweet.columns = ["Timestamp", "Picture_Link", "Twitter_ID", "Tweet_Text", "Tweet_link"]

In [32]:
pd.set_option('display.max_colwidth', None)
df_tweet.Tweet_Text.head(10)

0                                                                                                                                  RT @invest_answers: Crypto news, #Bitcoin Whales, flying motorbikes, wealth disparity, #FoodInflation covering #LUNA #AVAX #ADA #Tesla #Google and more live at noon PT  https://t.co/TUAG7xkc21 https://t.co/zesBmuo7CR
1                                                                                                                                   #Tesla tiene récord de autos vendidos. Es impresionante, pero no deja de sorprenderme que vendiendo 6 veces menos unidades que Toyota, por ejemplo, Tesla valga 3 o 4 veces más. https://t.co/u7Jm8oS54t vía @Inoreader
2    RT @CottonCodes: 🐒 #love in my #MariaCallas I KNOW Y #art HOLDonLINE #music cool ; #Impfpflicht #LauterbachRuecktritt mir egal #TT36® #women take #BREAKING #family MATTERS egal #alielali #ElonMusk #Dance THE #Tesla #haber https://t.co/GlfDe35SBk #community  #USA #Democracy #Hacking #SpyxFamily http

In [81]:
# create the text string
text = "RT @cb_doge: Tesla - A Trillion Dollar Company, World's Largest Car Manufacturer will be posting Q1,2022 Earnings on 4/20. \n\nDOGECOIN on their balance sheet will be made public for the first time. This will be just a beginning of the global adoption.\n\n#Tesla #Dogecoin @Tesla @elonmusk @omead https://t.co/Sd7t8vJ3Ii"

# define a regular expression pattern to match the different elements in the text
pattern = r'(RT\s)?(@\w+:\s)?([\w\s,.-]+)(#\w+)?(@\w+\s?)*([\w./:]+)?'

# use the regular expression pattern to extract the relevant information into separate variables
is_retweet = re.search(pattern, text).group(1) is not None
people_mentioned = re.findall(r'(@\w+)', text)
message = re.search(pattern, text).group(3)
hashtags = re.search(pattern, text).group(4)
other_links = re.search(pattern, text).group(6)

# create a dictionary with the extracted information
data = {'is_retweet': is_retweet,
        'people_mentioned': people_mentioned,
        'message': message,
        'hashtags': hashtags,
        'other_links': other_links}

# convert the dictionary into a DataFrame
df = pd.DataFrame(data)


In [82]:
df

,is_retweet,people_mentioned,message,hashtags,other_links
0,True,@cb_doge,"Tesla - A Trillion Dollar Company, World",None,None
1,True,@Tesla,"Tesla - A Trillion Dollar Company, World",None,None
2,True,@elonmusk,"Tesla - A Trillion Dollar Company, World",None,None
3,True,@omead,"Tesla - A Trillion Dollar Company, World",None,None


In [49]:
df_tweet["Timestamp_new"] = pd.to_datetime(df_tweet.Timestamp, format="%B %d, %Y at %I:%M%p",errors='coerce')

In [50]:
df_tweet.Timestamp_new[df_tweet.Timestamp_new.isnull()]

53988   NaT
72643   NaT
Name: Timestamp, dtype: datetime64[ns]

In [51]:
df_tweet.loc[53988,"Timestamp_new"] = pd.to_datetime(df_tweet.loc[53988,"Timestamp"], format="%d-%b-%y")
df_tweet.loc[72643,"Timestamp_new"] = pd.to_datetime(df_tweet.loc[72643,"Timestamp"], format="%d-%b-%y")

In [72]:
max_date = max(df_tweet.Timestamp_new)

In [73]:
min_date = min(df_tweet.Timestamp_new)

#### Explore stock dataframe

In [56]:
df_stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258 entries, 0 to 1257
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Date        1258 non-null   object
 1   Close/Last  1258 non-null   object
 2   Volume      1258 non-null   int64 
 3   Open        1258 non-null   object
 4   High        1258 non-null   object
 5   Low         1258 non-null   object
dtypes: int64(1), object(5)
memory usage: 59.1+ KB


In [57]:
df_stock.head(5)

,Date,Close/Last,Volume,Open,High,Low
0,03/02/2023,$190.90,181979200,$186.74,$193.7499,$186.01
1,03/01/2023,$202.77,156852800,$206.21,$207.20,$198.52
2,02/28/2023,$205.71,153144900,$210.59,$211.23,$203.75
3,02/27/2023,$207.63,161028300,$202.03,$209.42,$201.26
4,02/24/2023,$196.88,142228100,$196.325,$197.6695,$192.80


In [58]:
df_stock.Date = pd.to_datetime(df_stock.Date, format="%m/%d/%Y")

In [67]:
df_tweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152000 entries, 0 to 151999
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Timestamp      152000 non-null  object        
 1   Picture_Link   152000 non-null  object        
 2   Twitter_ID     152000 non-null  object        
 3   Tweet_Text     152000 non-null  object        
 4   Tweet_link     152000 non-null  object        
 5   Timestamp_new  152000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 7.0+ MB


In [76]:
df_stock_observed = df_stock[(df_stock.Date <= max_date) &(df_stock.Date >= min_date)]

In [78]:
print(max_date, min_date)
print(max(df_stock_observed.Date),min(df_stock_observed.Date))

2022-11-12 15:11:00 2022-04-10 19:44:00
2022-11-11 00:00:00 2022-04-11 00:00:00
